In [3]:
!pip install captcha

In [4]:
from captcha.image import ImageCaptcha
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt

CHARSETS = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

In [4]:
class CaptchaGenerator(object):
  
  def __init__(self, image_length):
    self.image_length = image_length
    self.image_captcha = ImageCaptcha()

  def get_random_text(self):
    retVal = ''
    for _ in range(self.image_length):
        retVal += CHARSETS[random.randint(0, len(CHARSETS)-1)]
    return retVal

  def get_labeled_image(self, samples):
    X, Y = [], []
    for _ in range(samples):
        text = self.get_random_text()
        image = self.image_captcha.generate(text, format='png')
        captcha_image = Image.open(image)
        array = np.array(captcha_image)
        X.append(array)
        Y.append(text)
        self.image_captcha.write(text, "samples/generated_captcha_images/%s.png" % text)
    return X, Y

In [23]:
gen = CaptchaGenerator(4)
gen.get_labeled_image(10000)

FileNotFoundError: [Errno 2] No such file or directory: 'samples/generated_captcha_images/COm5.png'

# Extract Single Letters From Captchas

In [5]:
import os
import os.path
import cv2
import glob
import imutils

def letters_segmentation(captcha_image_folder, output_folder):
  # Get a list of all the captcha images we need to process
  captcha_image_files = glob.glob(os.path.join(captcha_image_folder, "*"))
  counts = {}


  # loop over the image paths
  for (i, captcha_image_file) in enumerate(captcha_image_files):
      print("[INFO] processing image {}/{}".format(i + 1, len(captcha_image_files)))

      # Since the filename contains the captcha text (i.e. "2A2X.png" has the text "2A2X"),
      # grab the base filename as the text
      filename = os.path.basename(captcha_image_file)
      captcha_correct_text = os.path.splitext(filename)[0]
      
      # Load the image and convert it to grayscale
      image = cv2.imread(captcha_image_file)
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      # Remove some noise
      kernel = np.ones((3,3), np.uint8)
      gray = cv2.dilate(gray,kernel,iterations=1)
      gray = cv2.erode(gray,kernel,iterations=1)
      gray = cv2.fastNlMeansDenoising(gray, None,5, 7, 21)

      # threshold the image (convert it to pure black and white)
      thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

      # find the contours (continuous blobs of pixels) the image
      contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      letter_image_regions = []

      # Filter to select only contours letters and not noise
      for contour in contours:
          (x, y, w, h) = cv2.boundingRect(contour)
          if (w > 7 and h > 7):
            letter_image_regions.append((x, y, w, h))

      # If we found more or less than 4 letters in the captcha, our letter extraction
      # didn't work correcly. Skip the image instead of saving bad training data!
      if len(letter_image_regions) != 4:
          continue

      # Sort the detected letter images based on the x coordinate to make sure
      # we are processing them from left-to-right so we match the right image
      # with the right letter
      letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

      # Save out each letter as a single image
      for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_correct_text):
          # Grab the coordinates of the letter in the image
          x, y, w, h = letter_bounding_box

          # Extract the letter from the original image with a 2-pixel margin around the edge
          letter_image = gray[y: y + h , x: x + w]

          # Get the folder to save the image in
          save_path = os.path.join(output_folder, letter_text)

          # if the output directory does not exist, create it
          if not os.path.exists(save_path):
              os.makedirs(save_path)

          # write the letter image to a file
          count = counts.get(letter_text, 1)
          p = os.path.join(save_path, "{}.png".format(str(count).zfill(6)))
          cv2.imwrite(p, letter_image)

          # increment the count for the current key
          counts[letter_text] = count + 1


In [1]:
CAPTCHA_IMAGE_FOLDER = "samples/generated_captcha_images/"
OUTPUT_FOLDER = "samples/extracted_letter/"
# OUTPUT_FOLDER = "samples/extracted_letter_images/"

In [6]:
letters_segmentation(CAPTCHA_IMAGE_FOLDER, OUTPUT_FOLDER)

[INFO] processing image 1/13471
[INFO] processing image 2/13471
[INFO] processing image 3/13471
[INFO] processing image 4/13471
[INFO] processing image 5/13471
[INFO] processing image 6/13471
[INFO] processing image 7/13471
[INFO] processing image 8/13471
[INFO] processing image 9/13471
[INFO] processing image 10/13471
[INFO] processing image 11/13471
[INFO] processing image 12/13471
[INFO] processing image 13/13471
[INFO] processing image 14/13471
[INFO] processing image 15/13471
[INFO] processing image 16/13471
[INFO] processing image 17/13471
[INFO] processing image 18/13471
[INFO] processing image 19/13471
[INFO] processing image 20/13471
[INFO] processing image 21/13471
[INFO] processing image 22/13471
[INFO] processing image 23/13471
[INFO] processing image 24/13471
[INFO] processing image 25/13471
[INFO] processing image 26/13471
[INFO] processing image 27/13471
[INFO] processing image 28/13471
[INFO] processing image 29/13471
[INFO] processing image 30/13471
[INFO] processing i

[INFO] processing image 246/13471
[INFO] processing image 247/13471
[INFO] processing image 248/13471
[INFO] processing image 249/13471
[INFO] processing image 250/13471
[INFO] processing image 251/13471
[INFO] processing image 252/13471
[INFO] processing image 253/13471
[INFO] processing image 254/13471
[INFO] processing image 255/13471
[INFO] processing image 256/13471
[INFO] processing image 257/13471
[INFO] processing image 258/13471
[INFO] processing image 259/13471
[INFO] processing image 260/13471
[INFO] processing image 261/13471
[INFO] processing image 262/13471
[INFO] processing image 263/13471
[INFO] processing image 264/13471
[INFO] processing image 265/13471
[INFO] processing image 266/13471
[INFO] processing image 267/13471
[INFO] processing image 268/13471
[INFO] processing image 269/13471
[INFO] processing image 270/13471
[INFO] processing image 271/13471
[INFO] processing image 272/13471
[INFO] processing image 273/13471
[INFO] processing image 274/13471
[INFO] process

[INFO] processing image 487/13471
[INFO] processing image 488/13471
[INFO] processing image 489/13471
[INFO] processing image 490/13471
[INFO] processing image 491/13471
[INFO] processing image 492/13471
[INFO] processing image 493/13471
[INFO] processing image 494/13471
[INFO] processing image 495/13471
[INFO] processing image 496/13471
[INFO] processing image 497/13471
[INFO] processing image 498/13471
[INFO] processing image 499/13471
[INFO] processing image 500/13471
[INFO] processing image 501/13471
[INFO] processing image 502/13471
[INFO] processing image 503/13471
[INFO] processing image 504/13471
[INFO] processing image 505/13471
[INFO] processing image 506/13471
[INFO] processing image 507/13471
[INFO] processing image 508/13471
[INFO] processing image 509/13471
[INFO] processing image 510/13471
[INFO] processing image 511/13471
[INFO] processing image 512/13471
[INFO] processing image 513/13471
[INFO] processing image 514/13471
[INFO] processing image 515/13471
[INFO] process

[INFO] processing image 728/13471
[INFO] processing image 729/13471
[INFO] processing image 730/13471
[INFO] processing image 731/13471
[INFO] processing image 732/13471
[INFO] processing image 733/13471
[INFO] processing image 734/13471
[INFO] processing image 735/13471
[INFO] processing image 736/13471
[INFO] processing image 737/13471
[INFO] processing image 738/13471
[INFO] processing image 739/13471
[INFO] processing image 740/13471
[INFO] processing image 741/13471
[INFO] processing image 742/13471
[INFO] processing image 743/13471
[INFO] processing image 744/13471
[INFO] processing image 745/13471
[INFO] processing image 746/13471
[INFO] processing image 747/13471
[INFO] processing image 748/13471
[INFO] processing image 749/13471
[INFO] processing image 750/13471
[INFO] processing image 751/13471
[INFO] processing image 752/13471
[INFO] processing image 753/13471
[INFO] processing image 754/13471
[INFO] processing image 755/13471
[INFO] processing image 756/13471
[INFO] process

[INFO] processing image 969/13471
[INFO] processing image 970/13471
[INFO] processing image 971/13471
[INFO] processing image 972/13471
[INFO] processing image 973/13471
[INFO] processing image 974/13471
[INFO] processing image 975/13471
[INFO] processing image 976/13471
[INFO] processing image 977/13471
[INFO] processing image 978/13471
[INFO] processing image 979/13471
[INFO] processing image 980/13471
[INFO] processing image 981/13471
[INFO] processing image 982/13471
[INFO] processing image 983/13471
[INFO] processing image 984/13471
[INFO] processing image 985/13471
[INFO] processing image 986/13471
[INFO] processing image 987/13471
[INFO] processing image 988/13471
[INFO] processing image 989/13471
[INFO] processing image 990/13471
[INFO] processing image 991/13471
[INFO] processing image 992/13471
[INFO] processing image 993/13471
[INFO] processing image 994/13471
[INFO] processing image 995/13471
[INFO] processing image 996/13471
[INFO] processing image 997/13471
[INFO] process

...

[INFO] processing image 12476/13471
[INFO] processing image 12477/13471
[INFO] processing image 12478/13471
[INFO] processing image 12479/13471
[INFO] processing image 12480/13471
[INFO] processing image 12481/13471
[INFO] processing image 12482/13471
[INFO] processing image 12483/13471
[INFO] processing image 12484/13471
[INFO] processing image 12485/13471
[INFO] processing image 12486/13471
[INFO] processing image 12487/13471
[INFO] processing image 12488/13471
[INFO] processing image 12489/13471
[INFO] processing image 12490/13471
[INFO] processing image 12491/13471
[INFO] processing image 12492/13471
[INFO] processing image 12493/13471
[INFO] processing image 12494/13471
[INFO] processing image 12495/13471
[INFO] processing image 12496/13471
[INFO] processing image 12497/13471
[INFO] processing image 12498/13471
[INFO] processing image 12499/13471
[INFO] processing image 12500/13471
[INFO] processing image 12501/13471
[INFO] processing image 12502/13471
[INFO] processing image 1250

[INFO] processing image 12705/13471
[INFO] processing image 12706/13471
[INFO] processing image 12707/13471
[INFO] processing image 12708/13471
[INFO] processing image 12709/13471
[INFO] processing image 12710/13471
[INFO] processing image 12711/13471
[INFO] processing image 12712/13471
[INFO] processing image 12713/13471
[INFO] processing image 12714/13471
[INFO] processing image 12715/13471
[INFO] processing image 12716/13471
[INFO] processing image 12717/13471
[INFO] processing image 12718/13471
[INFO] processing image 12719/13471
[INFO] processing image 12720/13471
[INFO] processing image 12721/13471
[INFO] processing image 12722/13471
[INFO] processing image 12723/13471
[INFO] processing image 12724/13471
[INFO] processing image 12725/13471
[INFO] processing image 12726/13471
[INFO] processing image 12727/13471
[INFO] processing image 12728/13471
[INFO] processing image 12729/13471
[INFO] processing image 12730/13471
[INFO] processing image 12731/13471
[INFO] processing image 1273

[INFO] processing image 12934/13471
[INFO] processing image 12935/13471
[INFO] processing image 12936/13471
[INFO] processing image 12937/13471
[INFO] processing image 12938/13471
[INFO] processing image 12939/13471
[INFO] processing image 12940/13471
[INFO] processing image 12941/13471
[INFO] processing image 12942/13471
[INFO] processing image 12943/13471
[INFO] processing image 12944/13471
[INFO] processing image 12945/13471
[INFO] processing image 12946/13471
[INFO] processing image 12947/13471
[INFO] processing image 12948/13471
[INFO] processing image 12949/13471
[INFO] processing image 12950/13471
[INFO] processing image 12951/13471
[INFO] processing image 12952/13471
[INFO] processing image 12953/13471
[INFO] processing image 12954/13471
[INFO] processing image 12955/13471
[INFO] processing image 12956/13471
[INFO] processing image 12957/13471
[INFO] processing image 12958/13471
[INFO] processing image 12959/13471
[INFO] processing image 12960/13471
[INFO] processing image 1296

[INFO] processing image 13166/13471
[INFO] processing image 13167/13471
[INFO] processing image 13168/13471
[INFO] processing image 13169/13471
[INFO] processing image 13170/13471
[INFO] processing image 13171/13471
[INFO] processing image 13172/13471
[INFO] processing image 13173/13471
[INFO] processing image 13174/13471
[INFO] processing image 13175/13471
[INFO] processing image 13176/13471
[INFO] processing image 13177/13471
[INFO] processing image 13178/13471
[INFO] processing image 13179/13471
[INFO] processing image 13180/13471
[INFO] processing image 13181/13471
[INFO] processing image 13182/13471
[INFO] processing image 13183/13471
[INFO] processing image 13184/13471
[INFO] processing image 13185/13471
[INFO] processing image 13186/13471
[INFO] processing image 13187/13471
[INFO] processing image 13188/13471
[INFO] processing image 13189/13471
[INFO] processing image 13190/13471
[INFO] processing image 13191/13471
[INFO] processing image 13192/13471
[INFO] processing image 1319

[INFO] processing image 13397/13471
[INFO] processing image 13398/13471
[INFO] processing image 13399/13471
[INFO] processing image 13400/13471
[INFO] processing image 13401/13471
[INFO] processing image 13402/13471
[INFO] processing image 13403/13471
[INFO] processing image 13404/13471
[INFO] processing image 13405/13471
[INFO] processing image 13406/13471
[INFO] processing image 13407/13471
[INFO] processing image 13408/13471
[INFO] processing image 13409/13471
[INFO] processing image 13410/13471
[INFO] processing image 13411/13471
[INFO] processing image 13412/13471
[INFO] processing image 13413/13471
[INFO] processing image 13414/13471
[INFO] processing image 13415/13471
[INFO] processing image 13416/13471
[INFO] processing image 13417/13471
[INFO] processing image 13418/13471
[INFO] processing image 13419/13471
[INFO] processing image 13420/13471
[INFO] processing image 13421/13471
[INFO] processing image 13422/13471
[INFO] processing image 13423/13471
[INFO] processing image 1342

# EXTRACT LETTERS FROM ONE CAPTCHA STEP BY STEP


**What are contours ?**
Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity. The contours are a useful tool for shape analysis and object detection and recognition.

- For better accuracy, use binary images. So before finding contours, apply threshold or canny edge detection.
- Since OpenCV 3.2, findContours() no longer modifies the source image. In OpenCV, finding contours is like finding white object from black background. So remember, object to be found should be white and background should be black.

In [8]:
import cv2
import pickle
import os
import os.path
import glob
import imutils
import numpy as np
from imutils import paths
from tensorflow.keras import callbacks
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from keras.layers import Input
from keras.models import load_model

In [15]:
CAPTCHA_IMAGE_FOLDER = "samples/generated_captcha_images/"
LETTER_IMAGES_FOLDER = "samples/extracted_letter/"
MODEL_LABELS_FILENAME = "model_labels.dat"


# initialize the data and labels
data = []
labels = []
width = 40
height = 40
dim = (width, height)
for image_file in paths.list_images(LETTER_IMAGES_FOLDER):
  # Load the image and convert it to grayscale
  image = cv2.imread(image_file)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Resize the letter so it fits in a 40x40 pixel box
  image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

  # Add a third channel dimension to the image to make Keras happy
  image = np.expand_dims(image, axis=2)

  # Grab the name of the letter based on the folder it was in
  label = image_file.split(os.path.sep)[-2]

  # Add the letter image and it's label to our training data
  data.append(image)
  labels.append(label)

In [16]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [18]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, random_state=0)

In [19]:
lb = LabelBinarizer().fit(Y_train)
Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)
print("Train set size: %d" % len(X_train))
print("Test set size: %d" % len(X_test))

Train set size: 12858
Test set size: 4286


In [20]:
with open(MODEL_LABELS_FILENAME, "wb") as f:
      pickle.dump(lb, f)

In [22]:
model2 = Sequential()

# First convolutional layer with max pooling
model2.add(Conv2D(32, (5, 5), padding="same", input_shape=(40, 40, 1), activation="relu"))
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Second convolutional layer with max pooling
model2.add(Conv2D(48, (5, 5), padding="same", activation="relu"))
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Third convolutional layer with max pooling
model2.add(Conv2D(64, (5, 5), padding="same", activation="relu"))
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Hidden layer with 500 nodes
model2.add(Flatten())
model2.add(Dense(500, activation="relu"))

# Output layer with 32 nodes (one for each possible letter/number we predict)
model2.add(Dense(62, activation="softmax"))

# Ask Keras to build the TensorFlow model behind the scenes
model2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 40, 40, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 48)        38448     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 48)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 64)        76864     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)             

In [23]:
model2.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=10, verbose=1)

Epoch 1/10
402/402 [==============================] - 25s 61ms/step - loss: 1.7430 - accuracy: 0.5443 - val_loss: 0.5831 - val_accuracy: 0.8318
Epoch 2/10
402/402 [==============================] - 25s 62ms/step - loss: 0.4654 - accuracy: 0.8670 - val_loss: 0.4055 - val_accuracy: 0.8864
Epoch 3/10
402/402 [==============================] - 25s 61ms/step - loss: 0.3452 - accuracy: 0.8959 - val_loss: 0.3984 - val_accuracy: 0.8889
Epoch 4/10
402/402 [==============================] - 25s 62ms/step - loss: 0.2700 - accuracy: 0.9163 - val_loss: 0.3804 - val_accuracy: 0.8931
Epoch 5/10
402/402 [==============================] - 25s 61ms/step - loss: 0.2210 - accuracy: 0.9279 - val_loss: 0.3778 - val_accuracy: 0.9004
Epoch 6/10
402/402 [==============================] - 25s 62ms/step - loss: 0.1844 - accuracy: 0.9361 - val_loss: 0.4097 - val_accuracy: 0.8948
Epoch 7/10
402/402 [==============================] - 25s 61ms/step - loss: 0.1629 - accuracy: 0.9417 - val_loss: 0.3972 - val_accuracy:

In [24]:
from keras.applications import ResNet50
from tensorflow.keras import callbacks

In [25]:
input = Input(shape=(40, 40, 1))
resModel = ResNet50(input_tensor=input, include_top=False, weights=None, pooling='max')

output = resModel.output
output = Dense(62, activation="softmax")(output)

resultModel = Model(resModel.input, output)
resultModel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
resultModel.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40, 40, 1)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 46, 46, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 20, 20, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 20, 20, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [26]:
callbacks = [callbacks.TensorBoard('logs/trial_01', histogram_freq=1)]
history = resultModel.fit(
    X_train, Y_train, 
    validation_data=(X_test, Y_test), 
    batch_size=32, 
    epochs=10, 
    callbacks = callbacks,
    verbose=1)

Epoch 1/10
  1/402 [..............................] - ETA: 0s - loss: 8.6065 - accuracy: 0.0312WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
402/402 [==============================] - 389s 967ms/step - loss: 5.4919 - accuracy: 0.2732 - val_loss: 4.1949 - val_accuracy: 0.1703
Epoch 2/10
402/402 [==============================] - 377s 937ms/step - loss: 1.7492 - accuracy: 0.6464 - val_loss: 12.0426 - val_accuracy: 0.7072
Epoch 3/10
402/402 [==============================] - 371s 922ms/step - loss: 1.4908 - accuracy: 0.7094 - val_loss: 9.3402 - val_accuracy: 0.7042
Epoch 4/10
402/402 [==============================] - 367s 912ms/step - loss: 1.0144 - accuracy: 0.7878 - val_loss: 1.5508 - val_accuracy: 0.8066
Epoch 5/10
402/402 [=============================